In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import scipy.stats as stats

import util as u

In [2]:
vr = pd.read_csv('Data/video_review.csv')
pprd = pd.read_csv('Data/play_player_role_data.csv')
ppd = pd.read_csv('Data/player_punt_data.csv')
pi = pd.read_csv('Data/play_information.csv')
gd = pd.read_csv('Data/game_data.csv')

In [108]:
pos_cov = ['P','GL','GR','PPR','PC','PLW', 'PLT', 'PLG', 'PLS', 'PRG','PRT', 'PRW' ]
pos_ret = ['PR','PFB','VL', 'VRo','VRi','PLR','PLM','PLL','PDR1', 'PDR2', 'PDR3', 'PDL3', 'PDL2', 'PDL1']

In [17]:
vr.head()

,Season_Year,GameKey,PlayID,GSISID,Player_Activity_Derived,Turnover_Related,Primary_Impact_Type,Primary_Partner_GSISID,Primary_Partner_Activity_Derived,Friendly_Fire
0,2016,5,3129,31057,Tackling,No,Helmet-to-body,32482,Tackled,No
1,2016,21,2587,29343,Blocked,No,Helmet-to-helmet,31059,Blocking,No
2,2016,29,538,31023,Tackling,No,Helmet-to-body,31941,Tackled,No
3,2016,45,1212,33121,Tackling,No,Helmet-to-body,28249,Tackled,No
4,2016,54,1045,32444,Blocked,No,Helmet-to-body,31756,Blocked,Yes


In [45]:
len(vr)

37

### Merge df's to get player role in punt play and normal position

Injured player

In [112]:
df1 = pd.merge(vr, pprd.drop(columns='Season_Year'), how='left', on=['GameKey', 'PlayID', 'GSISID'], copy=False)

df1 = pd.merge(df1, ppd, how='left', on='GSISID', copy=False)


# there are duplicate entries in the ppd table -- player's number is discrepant between the entries. Drop one
df1.drop_duplicates(subset=['GameKey', 'PlayID', 'GSISID'], inplace=True)

df1['Team'] = df1['Role'].apply(lambda x: 'Coverage' if x in pos_cov else 'Return')

Primary Partner

In [116]:
vr['Primary_Partner_GSISID'] = pd.to_numeric(vr['Primary_Partner_GSISID'], errors='coerce')

df2 = pd.merge(vr, pprd.drop(columns='Season_Year'), how='left', left_on=['GameKey', 'PlayID', 'Primary_Partner_GSISID'], 
               right_on=['GameKey', 'PlayID', 'GSISID'], suffixes=('','_partner'), copy=False)

df2 = pd.merge(df2, ppd, how='left', left_on=['Primary_Partner_GSISID'],
                                right_on=['GSISID'], suffixes=('','_partner'), copy=False)

df2.drop(columns=['Season_Year', 'GSISID', 'Player_Activity_Derived',
       'Turnover_Related', 'Primary_Impact_Type', 
       'Primary_Partner_Activity_Derived', 'Friendly_Fire', 'GSISID_partner'], inplace=True)

# there are duplicate entries in the ppd table -- player's number is discrepant between the entries. Drop one
df2.drop_duplicates(subset=['GameKey', 'PlayID', 'Primary_Partner_GSISID'], inplace=True)

df2['Team'] = df2['Role'].apply(lambda x: 'Coverage' if x in pos_cov else 'Return')

Merge into one df

In [125]:
df3 = pd.merge(df1, df2, how='left', on=['GameKey', 'PlayID'], suffixes=('','_partner'), copy=False)

df3.drop(columns='Primary_Partner_GSISID_partner', inplace=True)

pi_cols_to_drop = ['Season_Year', 'Season_Type', 'Game_Date', 'Week',
       'Quarter', 'Play_Type', 'Poss_Team',
       'Home_Team_Visit_Team', 'PlayDescription']

df3 = pd.merge(df3, pi.drop(columns=pi_cols_to_drop), how='left', on=['GameKey', 'PlayID'], copy=False)

In [127]:
u.pkl_this('Output/injury_data_with_positions.pkl', df3)
# df1 = u.open_pkl('Output/injury_data_with_positions.pkl')